In [1]:
%pip install numpy
%pip install torch
%pip install timm
%pip install wandb


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
  Obtaining dependency information for timm from https://files.pythonhosted.org/packages/f0/1e/05287cb8984229d101874433df472b1fa3dcd6f746ccb6e8a26c7deeb1c7/timm-0.9.10-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.8/59.8 kB 1.7 MB/s eta 0:00:00
  Obtaining dependency information for torchvision from https://files.pythonhosted.org/packages/84/eb/4f6483ae9094e164dc5b9b792e377f7d37823b0bedc3eef3193d416d2bb6/torchvision-0.16.0-cp310-cp310-manylinux1_x86_64.whl.metadata
  Obtaining dependency information for pyyaml from https://files.pythonhosted.org/packages/29/61/bf33c6c85c55bc45a29eee3195848ff2d518d84735eb0e

## 70. 単語ベクトルの和による特徴量
問題50で構築した学習データ，検証データ，評価データを行列・ベクトルに変換したい．細かな仕様は問題を確認

In [46]:
import numpy as np
import re
import torch

from gensim.models import KeyedVectors
'''
torchの勉強がしたい
X : 事例の特徴ベクトルを並べた行列
    ・事例の特徴ベクトルとは
        ・事例（タイトル）の単語ベクトル列を平均したもの
        ・次元数は300
y : 正解ラベルを並べたベクトル
    ・4つのカテゴリ
        ・0: ビジネス
        ・1: 科学技術
        ・2: エンターテイメント
        ・3: 健康
このX, Yをtrain, valid, testに分割されたデータに対して作る
データの形式
    ・colum1: タイトル
    ・colum2: カテゴリ
        ・b: ビジネス
        ・t: 科学技術
        ・e: エンターテイメント
        ・m: 健康
'''

# データの読み込み
def generate_case(file_name):
    with open(file_name) as f:
        for line in f:
            yield line.rstrip().split('\t')

# 単語の特徴ベクトルへの変換
def word_to_feature(word):
    try:
        return word2vec_model[word]
    except:
        # print('not found the word: {}'.format(word))
        return np.zeros(300)

def preprocessing(text):
    text = re.sub(r'[\"\'.,:;\(\)#\|\*\+\!\?#$%&/\]\[\{\}]', '', text)
    text = re.sub(r'\s-\s', ' ', text)
    text = re.sub(r'\t', '', text)
    return text

# タイトルの特徴ベクトルへの変換→word_to_featureを平均したもの
def title_to_features(title):
    features = np.array([word_to_feature(word) for word in preprocessing(title).split(' ')]).mean(axis=0)
    return torch.from_numpy(features.astype(np.float32)).clone()

# カテゴリをラベルに変換
def category_to_label(category):
    labels = {'b': 0, 't': 1, 'e': 2, 'm': 3}
    return labels[category]

def save_data(X, Y, file_name):
    # torchで保存
    torch.save(X, file_name + '_X.pt')
    torch.save(Y, file_name + '_Y.pt')

def main(file_path):
    global word2vec_model
    word2vec_model = KeyedVectors.load_word2vec_format('/work01/s-nozue/100knock/chapter07/data/GoogleNews-vectors-negative300.bin', binary=True)
    X = []
    Y = []
    output_file_path = file_path.replace('.txt', '')
    count = 0

    for title, category in generate_case(file_path):
        X.append(title_to_features(title))
        Y.append(category_to_label(category))
        count += 1

    X = torch.stack(X)
    Y = torch.tensor(Y)
    save_data(X, Y, output_file_path)

ModuleNotFoundError: No module named 'gensim'

In [2]:
main('/work01/s-nozue/100knock/chapter08/datasets/valid_data.txt')

In [3]:
main('/work01/s-nozue/100knock/chapter08/datasets/test_data.txt')

In [4]:
main('/work01/s-nozue/100knock/chapter08/datasets/train_data.txt')

## 71. 単層ニューラルネットワークによる予測
問題70で保存した行列を読み込み，学習データについて以下の計算を実行せよ

In [15]:
import torch
import torch.nn as nn

def cal_softmax(x):
    softmax = nn.Softmax(dim=1)
    W = torch.randn(300, 4, requires_grad=True)
    xw = torch.matmul(x, W)
    return softmax(xw)

def main(file_path):
    X_train = torch.load(file_path)
    X_train1 = X_train[:1]
    X_train4 = X_train[:4]
    Y_hat1 = cal_softmax(X_train1)
    Y_hat4 = cal_softmax(X_train4)
    
    print('Y_hat1', Y_hat1, 'sum=', Y_hat1.sum(dim=1))
    print('Y_hat4', Y_hat4, 'sum=', Y_hat4.sum(dim=1))

In [188]:
main('/work01/s-nozue/100knock/chapter08/datasets/train_data_X.pt')

Y_hat1 tensor([[0.1914, 0.5460, 0.1230, 0.1396]], grad_fn=<SoftmaxBackward0>) sum= tensor([1.0000], grad_fn=<SumBackward1>)
Y_hat4 tensor([[0.0362, 0.1027, 0.8457, 0.0154],
        [0.3690, 0.4155, 0.1561, 0.0594],
        [0.0508, 0.0187, 0.8943, 0.0362],
        [0.3417, 0.1126, 0.3589, 0.1869]], grad_fn=<SoftmaxBackward0>) sum= tensor([1.0000, 1.0000, 1.0000, 1.0000], grad_fn=<SumBackward1>)


## 72. 損失と勾配の計算
学習データの事例x1と事例集合x1,x2,x3,x4に対して，クロスエントロピー損失と，行列Wに対する勾配を計算せよ．

In [47]:
import torch
import torch.nn as nn

def cal_cross_entropy_loss(y_hat, y):
    criterion = nn.CrossEntropyLoss()
    return criterion(y_hat, y)

def cal_gradient(y_hat, y, x):
    loss = cal_cross_entropy_loss(y_hat, y)
    print('loss:', loss)
    loss.backward()
    return x.grad

def main(X_file_path, Y_file_path):
    X_train = torch.load(X_file_path)
    Y_train = torch.load(Y_file_path)
    X_train1 = X_train[:1].requires_grad_()
    X_train4 = X_train[:4].requires_grad_()
    Y1 = Y_train[:1]
    Y4 = Y_train[:4]
    Y_hat1 = cal_softmax(X_train1)
    Y_hat4 = cal_softmax(X_train4)
    
    print('========== Y_hat1 ==========')
    print(Y_hat1, '->', torch.argmax(Y_hat1, dim=1), '"gold":', Y1)
    print('grad:', cal_gradient(Y_hat1, Y1, X_train1))

    print('========== Y_hat4 ==========')
    print( Y_hat4, '->', torch.argmax(Y_hat4, dim=1), '"gold":', Y4)
    print('grad:', cal_gradient(Y_hat4, Y4, X_train4))

In [191]:
main('/work01/s-nozue/100knock/chapter08/datasets/train_data_X.pt', '/work01/s-nozue/100knock/chapter08/datasets/train_data_Y.pt')

========== Y_hat1 ==========
tensor([[0.5703, 0.1023, 0.1865, 0.1409]], grad_fn=<SoftmaxBackward0>) -> tensor([0]) "gold": tensor([2])
loss: tensor(1.4685, grad_fn=<NllLossBackward0>)
grad: tensor([[-0.1775,  0.0140,  0.0729,  0.2089, -0.7135,  0.2121, -0.2800,  0.0240,
          0.1265,  0.3297,  0.1750,  0.1489, -0.2258,  0.2805,  0.2140, -0.2243,
         -0.0369, -0.0042, -0.0397, -0.0677, -0.3192, -0.0222, -0.2185, -0.2882,
          0.1209,  0.0011, -0.3448,  0.2535, -0.0590, -0.2058, -0.0021,  0.2254,
          0.0986, -0.2308, -0.1962, -0.0063, -0.0725,  0.3029,  0.1621, -0.1827,
         -0.0292, -0.0443, -0.2321, -0.1987, -0.3241,  0.3008,  0.0024, -0.3606,
          0.3592, -0.1538,  0.0013,  0.0766, -0.0933,  0.3255, -0.2350, -0.1914,
          0.1532, -0.0225, -0.0930,  0.0206, -0.0398,  0.1111, -0.2608,  0.3561,
          0.0799, -0.3846,  0.1436, -0.2042, -0.0719, -0.4149,  0.0688,  0.0871,
          0.0519,  0.0973, -0.0990,  0.0931, -0.0942,  0.1578, -0.2250,  0.1430,


## 73. 確率的勾配降下法による学習
確率的勾配降下法（SGD: Stochastic Gradient Descent）を用いて，行列Wを学習せよ．なお，学習は適当な基準で終了させればよい（例えば「100エポックで終了」など）．

In [73]:
import torch
import torch.nn as nn
import torch.optim as optim
import timm
import timm.scheduler
# import wandb

class NetWork(nn.Module):
    def __init__(self):
        super(NetWork, self).__init__()
        self.fc = nn.Linear(300, 4)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.fc(x)
        # x = self.softmax(x)
        return x

def main(X_file_path, Y_file_path):
    # wandb.init(project='100knock-2023')
    X_train = torch.load(X_file_path)
    Y_train = torch.load(Y_file_path)
    net = NetWork()
    optimizer = optim.SGD(net.parameters(), lr=0.001)
    scheduler = timm.scheduler.StepLRScheduler(optimizer, warmup_t=5, warmup_lr_init=0, decay_t=1, decay_rate = 0.5,)

    for epoch in range(10):
        for X, y in zip(X_train, Y_train):
            optimizer.zero_grad()
            output = net(X)
            loss = cal_cross_entropy_loss(output, y)
            loss.backward()
            optimizer.step()
        scheduler.step(epoch + 1)
        print(f'epoch: {epoch + 1}, loss: {loss.item()}')
        # wandb.log({'loss': loss.item(), 'lr': scheduler.get_lr()[0]})
    
    return net

In [74]:
net = main('/work01/s-nozue/100knock/chapter08/datasets/train_data_X.pt', '/work01/s-nozue/100knock/chapter08/datasets/train_data_Y.pt')

epoch: 1, loss: 1.3674858808517456
epoch: 2, loss: 1.0281847715377808
epoch: 3, loss: 0.73114013671875
epoch: 4, loss: 0.5134780406951904
epoch: 5, loss: 0.35844722390174866
epoch: 6, loss: 0.24946166574954987
epoch: 7, loss: 0.21100321412086487
epoch: 8, loss: 0.1950041800737381
epoch: 9, loss: 0.1877036988735199
epoch: 10, loss: 0.18421433866024017


## 74. 正解率の計測
問題73で求めた行列を用いて学習データおよび評価データの事例を分類したとき，その正解率をそれぞれ求めよ．

In [60]:
def eval(model, X_eval, Y_eval):
    with torch.no_grad():
        output = model(X_eval)
        loss = cal_cross_entropy_loss(output, Y_eval)
        accuracy = (torch.argmax(output, dim=1) == Y_eval).sum().item() / len(Y_eval)
        return loss, accuracy

In [62]:
X_eval = torch.load('/work01/s-nozue/100knock/chapter08/datasets/valid_data_X.pt')
Y_eval = torch.load('/work01/s-nozue/100knock/chapter08/datasets/valid_data_Y.pt')
loss, accuracy = eval(net, X_eval, Y_eval)
print(f'accuracy: {accuracy}')

accuracy: 0.781437125748503


## 75. 損失と正解率のプロット
問題73のコードを改変し，各エポックのパラメータ更新が完了するたびに，訓練データでの損失，正解率，検証データでの損失，正解率をグラフにプロットし，学習の進捗状況を確認できるようにせよ．

In [105]:
import torch
import torch.nn as nn
import torch.optim as optim
import wandb

class NetWork(nn.Module):
    def __init__(self):
        super(NetWork, self).__init__()
        self.fc = nn.Linear(300, 4)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.fc(x)
        # x = self.softmax(x)
        return x

def main(X_file_path, Y_file_path):
    wandb.init(project='100knock-2023')
    X_train = torch.load(X_file_path)
    Y_train = torch.load(Y_file_path)
    X_eval = torch.load('/work01/s-nozue/100knock/chapter08/datasets/valid_data_X.pt')
    Y_eval = torch.load('/work01/s-nozue/100knock/chapter08/datasets/valid_data_Y.pt')
    net = NetWork()
    optimizer = optim.SGD(net.parameters(), lr=0.01)
    scheduler = timm.scheduler.CosineLRScheduler(optimizer, warmup_t=5, warmup_lr_init=0, t_initial=20)

    for epoch in range(20):
        for X, y in zip(X_train, Y_train):
            optimizer.zero_grad()
            output = net(X)
            loss = cal_cross_entropy_loss(output, y)
            loss.backward()
            optimizer.step()
        eval_loss, acc = eval(net, X_eval, Y_eval)
        wandb.log({'train_loss': loss.item(), 'eval_loss': eval_loss.item(), 'accuracy': acc, 'lr:': optimizer.param_groups[0]["lr"]})
        print(f'epoch: {epoch + 1}, train_loss: {loss.item()}, eval_loss: {eval_loss}, accuracy: {acc}')
        scheduler.step(epoch + 1)

In [106]:
main('/work01/s-nozue/100knock/chapter08/datasets/train_data_X.pt', '/work01/s-nozue/100knock/chapter08/datasets/train_data_Y.pt')

accuracy,▁▇██████████████
eval_loss,█▆▅▄▃▂▂▂▂▁▁▁▁▁▁▁
lr:,▁▃▄▆███▇▆▆▅▄▄▃▃▂
train_loss,█▆▄▃▂▂▂▁▁▁▁▁▁▁▁▁
accuracy,0.78593
eval_loss,0.61378
lr:,0.00015
train_loss,0.08046


epoch: 1, train_loss: 1.3298213481903076, eval_loss: 1.3904500007629395, accuracy: 0.14520958083832336
epoch: 2, train_loss: 0.3355235457420349, eval_loss: 0.8342428803443909, accuracy: 0.782185628742515
epoch: 3, train_loss: 0.10837189853191376, eval_loss: 0.6497262716293335, accuracy: 0.7874251497005988
epoch: 4, train_loss: 0.04066023975610733, eval_loss: 0.5440211892127991, accuracy: 0.8106287425149701
epoch: 5, train_loss: 0.017862994223833084, eval_loss: 0.4744621217250824, accuracy: 0.843562874251497
epoch: 6, train_loss: 0.009713172912597656, eval_loss: 0.4322569966316223, accuracy: 0.8645209580838323
epoch: 7, train_loss: 0.006270141340792179, eval_loss: 0.40655645728111267, accuracy: 0.8697604790419161
epoch: 8, train_loss: 0.004493259359151125, eval_loss: 0.3894203007221222, accuracy: 0.8794910179640718
epoch: 9, train_loss: 0.003457641461864114, eval_loss: 0.37732452154159546, accuracy: 0.8824850299401198
epoch: 10, train_loss: 0.0028038020245730877, eval_loss: 0.3684832155

## 76. チェックポイント
問題75のコードを改変し，各エポックのパラメータ更新が完了するたびに，チェックポイント（学習途中のパラメータ（重み行列など）の値や最適化アルゴリズムの内部状態）をファイルに書き出せ

In [107]:
import torch
def save_checkpoint(model, optimizer, scheduler, file_name):
    checkpoint = {
        'model': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        'scheduler': scheduler.state_dict()
    }
    torch.save(checkpoint, file_name)

def main(X_file_path, Y_file_path):
    # wandb.init(project='100knock-2023')
    X_train = torch.load(X_file_path)
    Y_train = torch.load(Y_file_path)
    X_eval = torch.load('/work01/s-nozue/100knock/chapter08/datasets/valid_data_X.pt')
    Y_eval = torch.load('/work01/s-nozue/100knock/chapter08/datasets/valid_data_Y.pt')
    net = NetWork()
    optimizer = optim.SGD(net.parameters(), lr=0.01)
    scheduler = timm.scheduler.CosineLRScheduler(optimizer, warmup_t=5, warmup_lr_init=0, t_initial=20)

    for epoch in range(20):
        for X, y in zip(X_train, Y_train):
            optimizer.zero_grad()
            output = net(X)
            loss = cal_cross_entropy_loss(output, y)
            loss.backward()
            optimizer.step()
        eval_loss, acc = eval(net, X_eval, Y_eval)
        # wandb.log({'train_loss': loss.item(), 'eval_loss': eval_loss.item(), 'accuracy': acc, 'lr:': optimizer.param_groups[0]["lr"]})
        print(f'epoch: {epoch + 1}, train_loss: {loss.item()}, eval_loss: {eval_loss}, accuracy: {acc}')
        scheduler.step(epoch + 1)
        save_checkpoint(net, optimizer, scheduler, f'/work01/s-nozue/100knock/chapter08/models/checkpoint_{epoch + 1}.pt')

In [108]:
main('/work01/s-nozue/100knock/chapter08/datasets/train_data_X.pt', '/work01/s-nozue/100knock/chapter08/datasets/train_data_Y.pt')

epoch: 1, train_loss: 1.4502325057983398, eval_loss: 1.3994134664535522, accuracy: 0.14221556886227546
epoch: 2, train_loss: 0.371903657913208, eval_loss: 0.8329234719276428, accuracy: 0.7799401197604791
epoch: 3, train_loss: 0.1205444484949112, eval_loss: 0.6488741040229797, accuracy: 0.7844311377245509
epoch: 4, train_loss: 0.04491501674056053, eval_loss: 0.5432769656181335, accuracy: 0.8106287425149701
epoch: 5, train_loss: 0.019581111147999763, eval_loss: 0.4738258719444275, accuracy: 0.8473053892215568
epoch: 6, train_loss: 0.010583464987576008, eval_loss: 0.4317111670970917, accuracy: 0.8645209580838323
epoch: 7, train_loss: 0.006802614312618971, eval_loss: 0.4060709476470947, accuracy: 0.8705089820359282
epoch: 8, train_loss: 0.00485918577760458, eval_loss: 0.38897302746772766, accuracy: 0.8772455089820359
epoch: 9, train_loss: 0.003730245167389512, eval_loss: 0.3769018054008484, accuracy: 0.8817365269461078
epoch: 10, train_loss: 0.0030189435929059982, eval_loss: 0.368076652288

## 77. ミニバッチ化
問題76のコードを改変し，B事例ごとに損失・勾配を計算し，行列Wの値を更新せよ（ミニバッチ化）．Bの値を1,2,4,8,…と変化させながら，1エポックの学習に要する時間を比較せよ．

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import timm
import timm.scheduler
import time

class NetWork(nn.Module):
    def __init__(self):
        super(NetWork, self).__init__()
        self.fc = nn.Linear(300, 4)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.fc(x)
        # x = self.softmax(x)
        return x
    
def save_checkpoint(model, optimizer, scheduler, file_name):
    checkpoint = {
        'model': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        'scheduler': scheduler.state_dict()
    }
    torch.save(checkpoint, file_name)

def data_loader(x, y, batch_size):
    x_loader = torch.utils.data.DataLoader(x, batch_size=batch_size)
    y_loader = torch.utils.data.DataLoader(y, batch_size=batch_size)
    return x_loader, y_loader

def cal_cross_entropy_loss(y_hat, y):
    criterion = nn.CrossEntropyLoss()
    return criterion(y_hat, y)

def cal_gradient(y_hat, y, x):
    loss = cal_cross_entropy_loss(y_hat, y)
    print('loss:', loss)
    loss.backward()
    return x.grad

def eval(model, X_eval, Y_eval):
    with torch.no_grad():
        output = model(X_eval)
        loss = cal_cross_entropy_loss(output, Y_eval)
        accuracy = (torch.argmax(output, dim=1) == Y_eval).sum().item() / len(Y_eval)
        return loss, accuracy

def main(X_file_path, Y_file_path, batch_size=1):
    # wandb.init(project='100knock-2023')
    X_train = torch.load(X_file_path)
    Y_train = torch.load(Y_file_path)
    x_loader_train, y_loader_train = data_loader(X_train, Y_train, batch_size)

    X_eval = torch.load('/work01/s-nozue/100knock/chapter08/datasets/valid_data_X.pt')
    Y_eval = torch.load('/work01/s-nozue/100knock/chapter08/datasets/valid_data_Y.pt')
    # x_loader_eval, y_loader_eval = data_loader(X_eval, Y_eval, batch_size)
    
    net = NetWork()
    optimizer = optim.SGD(net.parameters(), lr=0.01)
    scheduler = timm.scheduler.CosineLRScheduler(optimizer, warmup_t=5, warmup_lr_init=0, t_initial=20)

    for epoch in range(1):
        start_time = time.time()
        for X, y in zip(x_loader_train, y_loader_train):
            optimizer.zero_grad()
            output = net(X)
            loss = cal_cross_entropy_loss(output, y)
            loss.backward()
            optimizer.step()
        end_time = time.time()
        print(f'epoch: {epoch + 1}, time: {end_time - start_time}')
        eval_loss, acc = eval(net, X_eval, Y_eval)
        # wandb.log({'train_loss': loss.item(), 'eval_loss': eval_loss.item(), 'accuracy': acc, 'lr:': optimizer.param_groups[0]["lr"]})
        print(f'epoch: {epoch + 1}, train_loss: {loss.item()}, eval_loss: {eval_loss}, accuracy: {acc}')
        scheduler.step(epoch + 1)
        save_checkpoint(net, optimizer, scheduler, f'/work01/s-nozue/100knock/chapter08/models/checkpoint_77_{epoch + 1}.pt')

/work01/s-nozue/100knock/chapter08/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
main('/work01/s-nozue/100knock/chapter08/datasets/train_data_X.pt', '/work01/s-nozue/100knock/chapter08/datasets/train_data_Y.pt', 1)

epoch: 1, time: 4.593250513076782
epoch: 1, train_loss: 1.3006435632705688, eval_loss: 1.3776284456253052, accuracy: 0.2657185628742515


In [15]:
main('/work01/s-nozue/100knock/chapter08/datasets/train_data_X.pt', '/work01/s-nozue/100knock/chapter08/datasets/train_data_Y.pt', 2)

epoch: 1, time: 2.686436414718628
epoch: 1, train_loss: 1.350126028060913, eval_loss: 1.3743897676467896, accuracy: 0.38772455089820357


In [16]:
main('/work01/s-nozue/100knock/chapter08/datasets/train_data_X.pt', '/work01/s-nozue/100knock/chapter08/datasets/train_data_Y.pt', 4)

epoch: 1, time: 1.5043866634368896
epoch: 1, train_loss: 1.395340919494629, eval_loss: 1.3925906419754028, accuracy: 0.2597305389221557


In [17]:
main('/work01/s-nozue/100knock/chapter08/datasets/train_data_X.pt', '/work01/s-nozue/100knock/chapter08/datasets/train_data_Y.pt', 8)

epoch: 1, time: 0.6768782138824463
epoch: 1, train_loss: 1.3528156280517578, eval_loss: 1.3854172229766846, accuracy: 0.19011976047904192


## 78. GPU上での学習
問題77のコードを改変し，GPU上で学習を実行せよ．

In [9]:
def main(X_file_path, Y_file_path, batch_size=1):
    # wandb.init(project='100knock-2023')
    X_train = torch.load(X_file_path)
    Y_train = torch.load(Y_file_path)
    x_loader_train, y_loader_train = data_loader(X_train, Y_train, batch_size)

    X_eval = torch.load('/work01/s-nozue/100knock/chapter08/datasets/valid_data_X.pt')
    Y_eval = torch.load('/work01/s-nozue/100knock/chapter08/datasets/valid_data_Y.pt')
    # x_loader_eval, y_loader_eval = data_loader(X_eval, Y_eval, batch_size)
    
    net = NetWork()
    optimizer = optim.SGD(net.parameters(), lr=0.01)
    scheduler = timm.scheduler.CosineLRScheduler(optimizer, warmup_t=5, warmup_lr_init=0, t_initial=20)

    # netをGPUに載せる 
    net.to('cuda')
    X_eval = X_eval.to('cuda')
    Y_eval = Y_eval.to('cuda')
    
    for epoch in range(10):
        start_time = time.time()
        for X, y in zip(x_loader_train, y_loader_train):
            X = X.to('cuda')
            y = y.to('cuda')
            optimizer.zero_grad()
            output = net(X)
            loss = cal_cross_entropy_loss(output, y)
            loss.backward()
            optimizer.step()
        end_time = time.time()
        print(f'epoch: {epoch + 1}, time: {end_time - start_time}')
        eval_loss, acc = eval(net, X_eval, Y_eval)
        # wandb.log({'train_loss': loss.item(), 'eval_loss': eval_loss.item(), 'accuracy': acc, 'lr:': optimizer.param_groups[0]["lr"]})
        print(f'epoch: {epoch + 1}, train_loss: {loss.item()}, eval_loss: {eval_loss}, accuracy: {acc}')
        scheduler.step(epoch + 1)
        save_checkpoint(net, optimizer, scheduler, f'/work01/s-nozue/100knock/chapter08/models/checkpoint_77_{epoch + 1}.pt')

In [10]:
main('/work01/s-nozue/100knock/chapter08/datasets/train_data_X.pt', '/work01/s-nozue/100knock/chapter08/datasets/train_data_Y.pt', 8)

epoch: 1, time: 1.7512094974517822
epoch: 1, train_loss: 1.3964546918869019, eval_loss: 1.3819477558135986, accuracy: 0.23802395209580837
epoch: 2, time: 1.7681827545166016
epoch: 2, train_loss: 1.182250738143921, eval_loss: 1.1406705379486084, accuracy: 0.7552395209580839
epoch: 3, time: 1.7972397804260254
epoch: 3, train_loss: 1.0387433767318726, eval_loss: 0.9954099655151367, accuracy: 0.7761976047904192
epoch: 4, time: 1.7793617248535156
epoch: 4, train_loss: 0.9156491756439209, eval_loss: 0.8854034543037415, accuracy: 0.7776946107784432
epoch: 5, time: 1.7892577648162842
epoch: 5, train_loss: 0.809401273727417, eval_loss: 0.7968606352806091, accuracy: 0.780688622754491
epoch: 6, time: 1.7966270446777344
epoch: 6, train_loss: 0.7329024076461792, eval_loss: 0.7354108691215515, accuracy: 0.781437125748503
epoch: 7, time: 1.7939023971557617
epoch: 7, train_loss: 0.6811598539352417, eval_loss: 0.6940565705299377, accuracy: 0.7829341317365269
epoch: 8, time: 1.7914259433746338
epoch: 8,

## 79. 多層ニューラルネットワーク
問題78のコードを改変し，バイアス項の導入や多層化など，ニューラルネットワークの形状を変更しながら，高性能なカテゴリ分類器を構築せよ

In [ ]:
import torch.nn.functional as F

class NetWork(nn.Module):
    def __init__(self):
        super(NetWork, self).__init__()
        self.fc0 = nn.Linear(300, 150)
        self.fc1 = nn.Linear(150, 100)
        self.fc2 = nn.Linear(100, 50)
        self.fc3 = nn.Linear(50, 4)
        self.activation = F.relu

    def forward(self, x):
        x = self.activation(self.fc0(x))
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.fc3(x)
        return x

def main(X_file_path, Y_file_path, batch_size=1):
    # wandb.init(project='100knock-2023')
    X_train = torch.load(X_file_path)
    Y_train = torch.load(Y_file_path)
    x_loader_train, y_loader_train = data_loader(X_train, Y_train, batch_size)

    X_eval = torch.load('/work01/s-nozue/100knock/chapter08/datasets/valid_data_X.pt')
    Y_eval = torch.load('/work01/s-nozue/100knock/chapter08/datasets/valid_data_Y.pt')
    # x_loader_eval, y_loader_eval = data_loader(X_eval, Y_eval, batch_size)
    
    net = NetWork()
    optimizer = optim.SGD(net.parameters(), lr=0.01)
    scheduler = timm.scheduler.CosineLRScheduler(optimizer, warmup_t=5, warmup_lr_init=0, t_initial=20)

    # netをGPUに載せる 
    net.to('cuda')
    X_eval = X_eval.to('cuda')
    Y_eval = Y_eval.to('cuda')
    
    for epoch in range(10):
        start_time = time.time()
        for X, y in zip(x_loader_train, y_loader_train):
            X = X.to('cuda')
            y = y.to('cuda')
            optimizer.zero_grad()
            output = net(X)
            loss = cal_cross_entropy_loss(output, y)
            loss.backward()
            optimizer.step()
        end_time = time.time()
        print(f'epoch: {epoch + 1}, time: {end_time - start_time}')
        eval_loss, acc = eval(net, X_eval, Y_eval)
        # wandb.log({'train_loss': loss.item(), 'eval_loss': eval_loss.item(), 'accuracy': acc, 'lr:': optimizer.param_groups[0]["lr"]})
        print(f'epoch: {epoch + 1}, train_loss: {loss.item()}, eval_loss: {eval_loss}, accuracy: {acc}')
        scheduler.step(epoch + 1)
        save_checkpoint(net, optimizer, scheduler, f'/work01/s-nozue/100knock/chapter08/models/checkpoint_77_{epoch + 1}.pt')